In [1]:
import pandas as pd
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
# 1. 读取数据
df = pd.read_csv('final_analysis_sample.csv')

In [2]:
outcomes_map = {
    '401(k) Assets':      ('d21ltaltb',       'taltb6'),
    'IRA Assets':         ('d21lthhira',      'thhira6'),
    'Other Fin. Assets':  ('d21lotherassets', 'otherassets6'),
    'Secured Debt':       ('d21lthhscdbt',    'thhscdbt6'),
    'Unsecured Debt':     ('d21lrhhuscbt',    'rhhuscbt6'),
    'Car Value':          ('d21ltcarval',     'tcarval6')
}

# 3. 定义 Treatment 和 Controls
treatment = 'temp'

# 动态获取控制变量
educ_cols = [c for c in df.columns if c.startswith('educ_')]
size_cols = [c for c in df.columns if c.startswith('tempallnew_')]
ind_cols = [c for c in df.columns if c.startswith('ind1dig_')]

base_covars = [
    'tage', 'thtotincyr1', 
    'taltb3', 'thhira3', 'otherassets3', 'tcarval3', 'thhscdbt3', 'rhhuscbt3',
    'taltb6', 'thhira6', 'otherassets6', 'tcarval6', 'thhscdbt6', 'rhhuscbt6',
    'efnp', 'esex', 'incmissing'
]

controls = base_covars + educ_cols + size_cols + ind_cols

X_cols = [treatment] + controls
df[X_cols] = df[X_cols].fillna(0).astype(float)

In [3]:
# =============================================================================
# 之前的数据准备代码 (保持不变，确保 df, treatment, controls, outcomes_map 已定义)
# =============================================================================
# (此处省略你提供的上半部分代码，假设已经运行完毕，df 和变量已准备好)

print("\n" + "="*125)
print("SECTION 5: SENSITIVITY ANALYSIS")
print("="*125)

# -----------------------------------------------------------------------------
# 5.1 Placebo Test (安慰剂检验)
# -----------------------------------------------------------------------------
# 逻辑：检验 Treatment 是否显著影响 "基准收入" (Year 1 Income)。
# 理论上，未来的资格 (Wave 9) 不应影响过去的收入 (Wave 6/Year 1)。
# 如果显著，说明存在 Selection Bias。
# -----------------------------------------------------------------------------

placebo_y_var = 'thtotincyr1'  # 使用基准年收入作为伪结果

# 从控制变量列表中移除作为 Y 的变量，避免共线性 (Y on Y)
placebo_controls = [c for c in controls if c != placebo_y_var]

# 准备数据
df_placebo = df.dropna(subset=[placebo_y_var]).copy()
X_placebo = sm.add_constant(df_placebo[[treatment] + placebo_controls])
y_placebo = df_placebo[placebo_y_var]

# 运行回归
model_placebo = sm.OLS(y_placebo, X_placebo).fit()

b_placebo = model_placebo.params[treatment]
p_placebo = model_placebo.pvalues[treatment]
se_placebo = model_placebo.bse[treatment]

print(f"\n[5.1] Placebo Test Results (Outcome: Baseline Income)")
print("-" * 60)
print(f"Outcome Variable: {placebo_y_var}")
print(f"Coefficient     : {b_placebo:.4f}")
print(f"Standard Error  : {se_placebo:.4f}")
print(f"P-value         : {p_placebo:.4f}")
if p_placebo > 0.10:
    print(">> PASS: Treatment effect is insignificant on placebo outcome.")
else:
    print(">> WARNING: Significant effect found on placebo outcome.")


SECTION 5: SENSITIVITY ANALYSIS

[5.1] Placebo Test Results (Outcome: Baseline Income)
------------------------------------------------------------
Outcome Variable: thtotincyr1
Coefficient     : 557.9892
Standard Error  : 2226.9046
P-value         : 0.8022
>> PASS: Treatment effect is insignificant on placebo outcome.


In [4]:
# -----------------------------------------------------------------------------
# 5.2 Robustness to Trimming (基于 Propensity Score 的修剪测试)
# -----------------------------------------------------------------------------
# 逻辑：计算倾向得分，逐步切除极端值 (Trimming)，看核心结果 (401k Assets) 是否稳定。
# -----------------------------------------------------------------------------
from sklearn.linear_model import LogisticRegression

print(f"\n[5.2] Sensitivity to Propensity Score Trimming (Outcome: 401(k) Assets)")
print("-" * 85)
print(f"{'Trim Range':<15} | {'N (Remaining)':<15} | {'Coef':<10} | {'SE':<8} | {'P-value':<8}")
print("-" * 85)

# 1. 估计 Propensity Score (Logit)
# 使用与主回归相同的控制变量
df_ps = df.copy()
X_ps = sm.add_constant(df_ps[controls])
y_ps = df_ps[treatment]

logit = LogisticRegression(max_iter=1000, solver='liblinear')
logit.fit(X_ps, y_ps)

# 将得分写入 DataFrame
df_ps['pscore'] = logit.predict_proba(X_ps)[:, 1]



[5.2] Sensitivity to Propensity Score Trimming (Outcome: 401(k) Assets)
-------------------------------------------------------------------------------------
Trim Range      | N (Remaining)   | Coef       | SE       | P-value 
-------------------------------------------------------------------------------------


In [5]:
sensitivity_targets = {
    '401(k) Assets': 'd21ltaltb',
    'IRA Assets':    'd21lthhira',
    'Car Value':     'd21ltcarval'
}
# 2. 定义核心结果变量
trim_levels = [0.0, 0.01, 0.05, 0.10, 0.15] # 切除两端 0%, 1%, 5%, 10%, 15%

print(f"\n[5.2] Sensitivity to Propensity Score Trimming (Multi-Outcome)")

for name, target_y in sensitivity_targets.items():
    print("\n" + "-" * 85)
    print(f"Outcome: {name}")
    print("-" * 85)
    print(f"{'Trim Range':<15} | {'N (Remaining)':<15} | {'Coef':<10} | {'SE':<8} | {'P-value':<8}")
    print("-" * 85)
    
    for trim in trim_levels:
        # 定义保留范围 [trim, 1-trim]
        mask = (df_ps['pscore'] >= trim) & (df_ps['pscore'] <= (1 - trim))
        df_trimmed = df_ps[mask].dropna(subset=[target_y])
        
        if len(df_trimmed) < 10: # 样本太少跳过
            continue
            
        # 在修剪后的样本上跑 OLS (With Controls)
        X_trim = sm.add_constant(df_trimmed[[treatment] + controls])
        y_trim = df_trimmed[target_y]
        
        model_trim = sm.OLS(y_trim, X_trim).fit()
        
        b_trim = model_trim.params[treatment]
        se_trim = model_trim.bse[treatment]
        p_trim = model_trim.pvalues[treatment]
        n_trim = int(model_trim.nobs)
        
        range_str = f"[{trim:.2f}, {1-trim:.2f}]"
        print(f"{range_str:<15} | {n_trim:<15} | {b_trim:>10.4f} | {se_trim:>8.4f} | {p_trim:>8.4f}")


[5.2] Sensitivity to Propensity Score Trimming (Multi-Outcome)

-------------------------------------------------------------------------------------
Outcome: 401(k) Assets
-------------------------------------------------------------------------------------
Trim Range      | N (Remaining)   | Coef       | SE       | P-value 
-------------------------------------------------------------------------------------
[0.00, 1.00]    | 835             |     0.8916 |   0.2923 |   0.0024
[0.01, 0.99]    | 833             |     0.8755 |   0.2924 |   0.0028
[0.05, 0.95]    | 827             |     0.8789 |   0.2921 |   0.0027
[0.10, 0.90]    | 818             |     0.8713 |   0.2926 |   0.0030
[0.15, 0.85]    | 814             |     0.8984 |   0.2916 |   0.0021

-------------------------------------------------------------------------------------
Outcome: IRA Assets
-------------------------------------------------------------------------------------
Trim Range      | N (Remaining)   | Coef       

In [7]:
# -----------------------------------------------------------------------------
# 5.3 E-value Analysis (Multi-Outcome: 401(k), IRA, Car)
# -----------------------------------------------------------------------------
# 逻辑：计算 E-value。
# - 对于正向系数 (beta > 0): RR = exp(beta)
# - 对于负向系数 (beta < 0): RR* = 1 / exp(beta)  (取倒数，转化为强度)
# 公式: E = RR + sqrt(RR * (RR - 1))
# -----------------------------------------------------------------------------
import numpy as np
# 定义要测试的三个变量
sensitivity_targets = {
    '401(k) Assets': 'd21ltaltb',
    'IRA Assets':    'd21lthhira',
    'Car Value':     'd21ltcarval'
}

print(f"\n[5.3] E-value Analysis (Method: VanderWeele & Ding, 2017)")
print("-" * 105)
header_e = f"{'Outcome':<15} | {'Coef':<8} | {'RR (Est)':<8} | {'E-val (Point)':<14} | {'E-val (CI Bound)':<16} | {'Significance'}"
print(header_e)
print("-" * 105)

for name, target_y in sensitivity_targets.items():
    
    # 准备数据运行回归
    reg_df = df.dropna(subset=[target_y])
    X_reg = sm.add_constant(reg_df[[treatment] + controls])
    model = sm.OLS(reg_df[target_y], X_reg).fit()
    
    beta = model.params[treatment]
    se = model.bse[treatment]
    pval = model.pvalues[treatment]
    
    # 判断显著性标记
    sig_mark = ""
    if pval < 0.01: sig_mark = "***"
    elif pval < 0.05: sig_mark = "**"
    elif pval < 0.1: sig_mark = "*"
    else: sig_mark = "NS" # Not Significant
    
    # --- E-value 计算逻辑 ---
    
    # Case 1: 正向效应 (如 401k, IRA)
    if beta >= 0:
        rr_est = np.exp(beta)
        
        # Point Estimate E-value
        if rr_est > 1:
            eval_point = rr_est + np.sqrt(rr_est * (rr_est - 1))
        else:
            eval_point = 1.0
            
        # CI Lower Bound E-value (保守估计: 看 CI 下限是否 > 0)
        ci_lower = beta - 1.96 * se
        rr_lower = np.exp(ci_lower)
        if rr_lower > 1:
            eval_ci = rr_lower + np.sqrt(rr_lower * (rr_lower - 1))
        else:
            eval_ci = 1.0 # 如果 CI 包含 0，则无法拒绝 Null，E-value 为 1
            
    # Case 2: 负向效应 (如 Car Value)
    else:
        # 对于负数，Risk Ratio < 1，我们需要取倒数来看看"强度"
        # 比如系数 -0.4 -> RR = 0.67 -> 我们关心的是 1/0.67 = 1.49 倍的强度
        rr_raw = np.exp(beta)
        rr_est = 1.0 / rr_raw 
        
        # Point Estimate E-value
        if rr_est > 1:
            eval_point = rr_est + np.sqrt(rr_est * (rr_est - 1))
        else:
            eval_point = 1.0
            
        # CI Upper Bound E-value (对于负数，上限最接近 0)
        ci_upper = beta + 1.96 * se
        # 如果上限依然小于 0 (显著负)，则计算 E-value
        if ci_upper < 0:
            rr_upper_inv = 1.0 / np.exp(ci_upper)
            eval_ci = rr_upper_inv + np.sqrt(rr_upper_inv * (rr_upper_inv - 1))
        else:
            eval_ci = 1.0 # 如果 CI 穿过 0，E-value 为 1

    print(f"{name:<15} | {beta:>8.4f} | {rr_est:>8.4f} | {eval_point:>14.4f} | {eval_ci:>16.4f} | {sig_mark}")

print("-" * 105)
print("Notes:")
print("1. RR (Est): Implied Risk Ratio. For negative coefs, RR = 1/exp(beta).")
print("2. E-val (CI Bound): The E-value for the confidence interval limit closest to the null.")
print("   - If result is not significant (NS), E-val (CI) is 1.0 by definition.")
print("3. Interpretation: Higher E-value = Stronger robustness to unobserved confounding.")
print("="*125)


[5.3] E-value Analysis (Method: VanderWeele & Ding, 2017)
---------------------------------------------------------------------------------------------------------
Outcome         | Coef     | RR (Est) | E-val (Point)  | E-val (CI Bound) | Significance
---------------------------------------------------------------------------------------------------------
401(k) Assets   |   0.8916 |   2.4391 |         4.3126 |           2.0937 | ***
IRA Assets      |   0.3621 |   1.4363 |         2.2280 |           1.0000 | NS
Car Value       |  -0.3663 |   1.4423 |         2.2411 |           1.0000 | NS
---------------------------------------------------------------------------------------------------------
Notes:
1. RR (Est): Implied Risk Ratio. For negative coefs, RR = 1/exp(beta).
2. E-val (CI Bound): The E-value for the confidence interval limit closest to the null.
   - If result is not significant (NS), E-val (CI) is 1.0 by definition.
3. Interpretation: Higher E-value = Stronger robustness t